In [180]:
import torchvision.models as models
import torch

model_dict = {
    'resnet18': models.resnet18(weights=None), # ++ 
    'alexnet' : models.alexnet(weights=None), # Error
    'vgg16_bn': models.vgg16_bn(weights=None), # Memory
    'vgg16' : models.vgg16(weights=None), # RuntimeError: Detected the following values in `preds`: tensor([111, 530, 626, 644, 818], device='cuda:0') but expected only the following values [0,1] since `preds` is a label tensor.
    'vgg19_bn': models.vgg19_bn(weights=None), # Memory
    'vgg19': models.vgg19(weights=None), # RuntimeError: Detected the following values in `preds`: tensor([111, 530, 644, 818], device='cuda:0') but expected only the following values [0,1] since `preds` is a label tensor.
    'squeezenet' : models.squeezenet1_0(weights=None), # 
    'densenet' : models.densenet161(weights=None), # 
    'inception_v3' : models.inception_v3(weights=None), # 
    'googlenet' : models.googlenet(weights=None), # 
    'shufflenet' : models.shufflenet_v2_x1_0(weights=None), # 
    'mobilenet' : models.mobilenet_v2(weights=None), # 
    'resnext50_32x4d' : models.resnext50_32x4d(weights=None), # 
    'wide_resnet50_2' : models.wide_resnet50_2(weights=None), # ++ (32bs = memory error)
    'mnasnet' : models.mnasnet1_0(weights=None), # ...
}

model_names = {
'resnet18': 'fc',
'alexnet': 'classifier',
'vgg16_bn': 'classifier',
'vgg16': 'classifier',
'vgg19_bn': 'classifier',
'vgg19': 'classifier',
'squeezenet': 'classifier',
'densenet': 'classifier',
'inception_v3': 'fc',
'googlenet': 'fc',
'shufflenet': 'fc',
'mobilenet': 'classifier',
'resnext50_32x4d': 'fc',
'wide_resnet50_2': 'fc',
'mnasnet': 'classifier'
}
num_classes = 2

c:\Users\user\anaconda3\envs\breakkit\lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
c:\Users\user\anaconda3\envs\breakkit\lib\site-packages\torchvision\models\googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [181]:
for model_name, model in model_dict.items():
    # Get last layer.
    last_layer_attr = model_names[model_name]
    last_layer = getattr(model, last_layer_attr)

    if isinstance(last_layer, torch.nn.Linear):
        num_ftrs = last_layer.in_features
        setattr(model, last_layer_attr, torch.nn.Linear(num_ftrs, num_classes))
    elif isinstance(last_layer, torch.nn.Sequential):
        if model_name != 'squeezenet':
            last_layer_ = last_layer[-1]
            num_ftrs = last_layer_.in_features
            model.classifier[-1] = torch.nn.Linear(num_ftrs, num_classes)
        else:
            last_layer = model.classifier
            last_layer_ = last_layer[1]
            num_ftrs = last_layer_.in_channels
            kernel_size = last_layer_.kernel_size
            model.classifier[1] = torch.nn.Conv2d(num_ftrs, num_classes, kernel_size)
    # Update the last layer.
    model_dict[model_name] = model

In [133]:
num_channels = 4
if num_channels != 3:
    for model_name, model in model_dict.items():
        if model_name in ['resnet18', 'resnext50_32x4d', 'wide_resnet50_2']:
            model.conv1.in_channels = num_channels
        elif model_name in ['alexnet', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'vgg16', 'densenet', 'squeezenet']:
            model.features[0].in_channels = num_channels
        elif model_name in ['mnasnet']:
            model.layers[0].in_channels = num_channels
        elif model_name == 'shufflenet':
            model.conv1[0].in_channels = 4
        elif model_name == 'inception_v3':
            model.Conv2d_1a_3x3.conv.in_channels = 4
        elif model_name == 'googlenet':
            model.conv1.conv.in_channels = 4
    # Update the dictionary.
    model_dict[model_name] = model

In [177]:
# Define a simple convolutional layer
conv = torch.nn.Conv2d(4, 64, kernel_size=3, stride=1, padding=1)

# Check the initial weights before applying initialization
print("Initial weights:")
print(conv.weight.shape)
new_weights = conv.weight.data

Initial weights:
torch.Size([64, 4, 3, 3])


In [184]:
model = model_dict['mobilenet']
model.features[0][0]

Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

In [158]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Add the parent directory to the Python path
sys.path.append(parent_dir)
from tools import BreaKHis

import torch
import torch.nn as nn
from torch import optim
from torchvision import transforms as T
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import WeightedRandomSampler, random_split, RandomSampler
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from models.fpcn import FPCN
from models.utilities.losses import FocalLoss
import os, sys
import warnings 

In [159]:
myDataset = BreaKHis(
    root='C:/Users/user/Dersler/Machine and Deep Learning/Project/BreaKHis_v1/',
                transform = T.Compose([
                T.ToPILImage(),  # Convert numpy.ndarray to PIL Image
                T.Resize(256),
                T.CenterCrop(224),
                T.ToTensor(),
                T.Normalize(mean=[0, 0 ,0 ,0], std=[0, 0 ,0 ,0])
            ]),
                mode = 'binary',
                imageLikefeatures = ['hog']
                )

100%|██████████| 1370/1370 [00:09<00:00, 140.29it/s]
h: 0it [00:00, ?it/s]

??


IndexError: tuple index out of range